In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from scipy.io import loadmat
from joblib import Parallel, delayed

In [2]:
class TrajStartEnd():
    def __init__(self, obj_name, path = '/datasets/eventdata/batch2/'):
        self.path = path
        self.obj_name = obj_name
        self.obj_path = self.path + 'traj_start_ends/' + obj_name + '.startend'
        self.traj_start_end = np.array(open(self.obj_path, "r").read().split(" ")).astype(float) 

In [3]:
class TactileData():
    def __init__(self, obj_name, selection="full", path = '/datasets/eventdata/batch2/'):
        self.path = path
        self.obj_name = obj_name
#         self.path2 = traj_start_end_path
        self.file_path_tac = self.path + 'aces_recordings/' + obj_name + '.tact'
    
        trajStartEnd = TrajStartEnd(obj_name, path)
        self.traj_start_end = trajStartEnd.traj_start_end
        
        # read tact file
        df = pd.read_csv(self.file_path_tac, delimiter=' ', names=['polarity', 'cell_index', 'timestamp_sec', 'timestamp_nsec'], dtype=int)
        df = df.assign(timestamp=df.timestamp_sec + df.timestamp_nsec/1000000000)
        df = df.drop(['timestamp_sec', 'timestamp_nsec'], axis=1)

        # timestamp start from zero
#         print(self.traj_start_end[1], df.timestamp[0])
        df.timestamp = df.timestamp - self.traj_start_end[1]
#         print(self.traj_start_end[1], df.timestamp[0])
        if selection == 'full':
            self.start_t = 0.0
            _T = 8.5
        elif selection == 'grasp':
            self.start_t = self.traj_start_end[3] - self.traj_start_end[1]
            _T = 4.0
        elif selection == 'lift':
            self.start_t = self.traj_start_end[4] - self.traj_start_end[1]
            _T = 0.75
        elif selection == 'hold':
            self.start_t = self.traj_start_end[4] + 2.0 - self.traj_start_end[1]
            _T = 1.25
        elif selection == 'lift_hold':
            self.start_t = self.traj_start_end[4] - self.traj_start_end[1]
            _T = 2.0
        elif selection == 'grasp_lift':
            self.start_t = self.traj_start_end[3] - self.traj_start_end[1]
            _T = 4.75
        elif selection == 'grasp_lift_hold':
            self.start_t = self.traj_start_end[3] - self.traj_start_end[1]
            _T = 6.5
        
        self.df = df
#         print(self.start_t, self.start_t + _T)
        #self.df = df[(df.timestamp >= mask_start) & (df.timestamp <= mask_end)]
        #self.df.timestamp = self.df.timestamp
        # Cells are not used: 17 and 49
        
        # binarization
        self.T = _T #4 #0.75 # 9 # 20
        self.bin_duration = 0.001
        self.threshold = 1
        self.bin_number = int(np.floor(self.T/self.bin_duration))
        self.binarized=False
        self.data_matrix = np.zeros([80,2,self.bin_number], dtype=int)
        
    def binarize(self, bin_duration = 0.002):
        
        # check if already binarized
        if self.binarized and self.bin_duration == bin_duration:
            return self.data_matrix
        
        self.bin_duration = bin_duration
        self.bin_number = int(np.floor(self.T/self.bin_duration))
#         print(self.T/self.bin_duration, self.bin_number)
        self.data_matrix = np.zeros([80,2,self.bin_number], dtype=int)
        
        pos_df = self.df[self.df.polarity==1]
        neg_df = self.df[self.df.polarity==0]
        
        end_t = self.start_t + self.bin_duration
        count = 0
        
        init_t = self.start_t
        
        while(end_t <= self.T + init_t): # start_t <= self.T
#             print('Its:', count, self.start_t, end_t)
        
            _pos_count = pos_df[( (pos_df.timestamp >= self.start_t ) & (pos_df.timestamp < end_t ) )]
            _pos_selective_cells = _pos_count.cell_index.value_counts() > self.threshold
            if len(_pos_selective_cells):
                self.data_matrix[_pos_selective_cells[_pos_selective_cells].index.values - 1, 0, count] = 1
                
            _neg_count = neg_df[( (neg_df.timestamp >= self.start_t ) & (neg_df.timestamp < end_t ) )]
            _neg_selective_cells = _neg_count.cell_index.value_counts() > self.threshold
            if len(_neg_selective_cells):
                self.data_matrix[_neg_selective_cells[_neg_selective_cells].index.values - 1, 1, count] = 1

            self.start_t = end_t
            end_t += self.bin_duration
            count += 1
            
        self.binarized = True
        
        self.data_matrix = np.delete(self.data_matrix, [16, 48], 0)
        return self.data_matrix
    
    def clean(self):
        self.data_matrix=[]
        self.binarized=False

In [4]:
class CameraData():
    def __init__(self, obj_name, selection="full", path = '/datasets/eventdata/batch2/'):
        self.path = path

        # propophesee hyperparameters
        self.c = 2
        self.w = 200#300
        self.h = 250#350      
        x0 = 180#230#180
        y0 = 0#100#0
        file_path = path + "prophesee_recordings/" + obj_name #+ ".start" # _td.mat
        start_time = float(open(file_path + ".start", "r").read())
        
        trajStartEnd = TrajStartEnd(obj_name, path)
        self.traj_start_end = trajStartEnd.traj_start_end
        delta_time = start_time - self.traj_start_end[0]
        self.traj_start_end += delta_time
        self.traj_start_end = self.traj_start_end - self.traj_start_end[0]
        
        if selection == 'full':
            self.start_t = self.traj_start_end[1]
            _T = 8.5
        elif selection == 'reaching':
            self.start_t = self.traj_start_end[1]
            _T = 2.0
        elif selection == 'grasp':
            self.start_t = self.traj_start_end[3]
            _T = 4.0
        elif selection == 'lift':
            self.start_t = self.traj_start_end[4]
            _T = 0.75
        elif selection == 'hold':
            self.start_t = self.traj_start_end[4] + 2.0
            _T = 1.25
        elif selection == 'lift_hold':
            self.start_t = self.traj_start_end[4]
            _T = 2.0
        elif selection == 'grasp_lift':
            self.start_t = self.traj_start_end[3]
            _T = 4.75
        elif selection == 'grasp_lift_hold':
            self.start_t = self.traj_start_end[3]
            _T = 6.5
        
        td_data = loadmat(file_path + "_td.mat")['td_data']
        df=pd.DataFrame(columns=['x', 'y', 'polarity', 'timestamp'])
        a = td_data['x'][0][0]
        b = td_data['y'][0][0]

        mask_y = (b >= 100)
        mask_x = (a >= 230) & (a < 430)
        a1 = a[ mask_x & mask_y ] - 230
        b1 = b[ mask_x & mask_y ] - 100
        df.x = a1.flatten()
        df.y = b1.flatten()
#         df.x = td_data['x'][0][0].flatten() - x0 # x coordinate
#         df.y = td_data['y'][0][0].flatten() - y0 # y coordinate
        df.polarity = td_data['p'][0][0][ mask_x & mask_y ].flatten() # polarity with value -1 or 1
        df.timestamp = td_data['ts'][0][0][ mask_x & mask_y ].flatten()/1000000.0 # spiking time in microseconds, convert to seconds
        df = df.reset_index()
        self.df = df
        
        self.T = _T #20
        self.bin_duration = -1.0 #0.002
        self.bin_number = -1 #int(np.floor(self.T/self.bin_duration))
        self.binarized = False
        
        self.data_matrix = []#np.zeros([self.c, self.w, self.h, self.bin_number], dtype=int)
        self.threshold = 1
        
    def binarize(self, bin_duration = 0.02):
        
        self.bin_duration = bin_duration
        self.bin_number = int(np.floor(self.T/self.bin_duration))
#         print(self.T/self.bin_duration, self.bin_number, self.T)
        self.data_matrix = np.zeros([self.c, self.w, self.h, self.bin_number], dtype=int)
        #print(self.data_matrix.shape)
        # self.data_matrix = np.zeros([self.c, 250, 200, self.bin_number], dtype=int)
        
        pos_df = self.df[self.df.polarity==1]
        neg_df = self.df[self.df.polarity==-1]
        #print(pos_df.shape, neg_df.shape)
        
        end_t = self.start_t + self.bin_duration
        count = 0
        
        init_t = self.start_t
        
        while(end_t <= self.T + init_t): # start_t <= self.T
#             print('Its:', count, self.start_t, end_t)
        
            _pos_count = pos_df[( (pos_df.timestamp >= self.start_t ) & (pos_df.timestamp < end_t ) )]
#             print(_pos_count.shape)

            b = pd.DataFrame(index=_pos_count.index)
            #print(_pos_count['y'])
            b = b.assign(xy = _pos_count['x'].astype(str) + '_' + _pos_count['y'].astype(str))
            mask = b.xy.value_counts() >= self.threshold
            some_array = mask[mask].index.values.astype(str)
            #print(some_array)
            xy = np.array(list(map(lambda x: x.split('_'), some_array))).astype(int)
            if xy.shape[0] > 0:
                self.data_matrix[0,xy[:,0],
                                   xy[:,1], count] = 1
                
            _neg_count = neg_df[( (neg_df.timestamp >= self.start_t ) & (neg_df.timestamp < end_t ) )]
            b = pd.DataFrame(index=_neg_count.index)
            b = b.assign(xy = _neg_count['x'].astype(str) + '_' + _neg_count['y'].astype(str))
            mask = b.xy.value_counts() >= self.threshold
            some_array = mask[mask].index.values.astype(str)
            xy = np.array(list(map(lambda x: x.split('_'), some_array))).astype(int)
            if xy.shape[0] > 0:
                self.data_matrix[1,xy[:,0],
                                  xy[:,1], count] = 1
            # print(_pos_count.shape[0] + _neg_count.shape[0])
            


            self.start_t = end_t
            end_t += self.bin_duration
            count += 1
            
        self.binarized = True
        self.data_matrix = np.swapaxes(self.data_matrix, 1,2)
        
        return self.data_matrix
    
    def clean(self):
        self.data_matrix=[]
        self.binarized=False

In [5]:
#file_path = '/datasets/eventdata/batch2/' + "prophesee_recordings/" + '107-a_pepsi_bottle_01'

In [6]:
#td_data = loadmat(file_path + "_td.mat")['td_data']

In [7]:
#td_data['x'][0][0].shape, td_data['y'][0][0].shape

In [8]:
# a = td_data['x'][0][0]
# b = td_data['y'][0][0]

# mask_y = (b >= 100)
# mask_x = (a >= 230) & (a < 430)

# td_data['x'][0][0][ mask_x & mask_y ].shape
# td_data['y'][0][0][ mask_x & mask_y ].shape

In [9]:
# np.unique(td_data['x'][0][0][ mask_x & mask_y ]- 230)
# np.unique(td_data['y'][0][0][ mask_x & mask_y ]- 100)

In [10]:
# a.flatten().shape

In [11]:
# b[(b >= 100)]-100

In [12]:
# a[(a >= 230) & (a < 430)] - 230

In [13]:
# aaa = CameraData('107-a_pepsi_bottle_01', "grasp_lift_hold", '/datasets/eventdata/batch2/')

In [14]:
#np.unique(aaa.df.x)

In [15]:
#np.unique(aaa.df.y)

In [16]:
# bbb = aaa.binarize(0.02)

In [17]:
def tact_bin_save(file_name, overall_count, bin_duration, path, selection, save_dir):
    tac_data = TactileData(file_name, selection, path)
    tacData = tac_data.binarize(bin_duration)
    tac_data.clean()
    np.save(save_dir + str(overall_count) + '_tact.npy', tacData.astype(np.uint8))

In [18]:
def vis_bin_save(file_name, overall_count, bin_duration, path, selection, save_dir):
    cam_data = CameraData(file_name, selection, path)
    visData = cam_data.binarize(bin_duration)
    np.save(save_dir + str(overall_count) + '_vis.npy', visData.astype(np.uint8))
    cam_data.clean()

In [19]:
class ViTacData():
    def __init__(self, ViTacDataPath, save_dir, list_of_objects, selection="full"):
        self.path = ViTacDataPath
        self.list_of_objects = list_of_objects
        self.iters = 20
        self.bin_duration = 0.002
        self.save_dir = save_dir #
        self.selection = selection
        
    def vis_bin_save(self, file_name, overall_count, bd):
        cam_data = CameraData(file_name, self.selection, self.path)
        visData = cam_data.binarize(self.bin_duration)
        np.save(self.save_dir + str(overall_count) + '_vis.npy', visData.astype(np.uint8))
        cam_data.clean()
        
    def binarize_save(self, bin_duration =0.002, modality=2):
        self.bin_duration = bin_duration
        'saves binarized tactile and prophesee data'
        current_label = 0
        overall_count = 0
        big_list_tact = []
        big_list_vis = []
        for obj in self.list_of_objects:
            print(obj, ' being processed ...')
            for i in range(1, self.iters+1): #tqdm(range(1, self.iters+1)):
                
                i_str = str(i)
                if i < 10:
                    i_str = '0' + i_str
                
                file_name = obj + '_' + i_str
                # tactile
                if modality == 2 or modality == 0:
                    big_list_tact.append([file_name, overall_count, self.bin_duration,
                                     self.path, self.selection, self.save_dir])

                if modality == 2 or modality == 1:
                    big_list_vis.append([file_name, overall_count, self.bin_duration,
                                     self.path, self.selection, self.save_dir])
#                     cam_data = CameraData(file_name, self.selection, self.path)
#                     visData = cam_data.binarize(self.bin_duration)
#                     np.save(self.save_dir + str(overall_count) + '_vis.npy', visData.astype(np.uint8))
#                     cam_data.clean()
                overall_count+=1
            current_label+=1
        if modality == 2 or modality == 0:
            Parallel(n_jobs=18)(delayed(tact_bin_save)(zz[0], zz[1], zz[2], zz[3], zz[4], zz[5]) for zz in big_list_tact)
            print('Tactile has done')
        if modality == 2 or modality == 1:
            Parallel(n_jobs=10)(delayed(vis_bin_save)(zz[0], zz[1], zz[2], zz[3], zz[4], zz[5]) for zz in big_list_vis)

In [20]:
# batch2
list_of_objects2 = [
    '107-a_pepsi_bottle',
    '107-b_pepsi_bottle',
    '107-c_pepsi_bottle',
    '107-d_pepsi_bottle',
    '107-e_pepsi_bottle',
    '108-a_tuna_fish_can',
    '108-b_tuna_fish_can',
    '108-c_tuna_fish_can',
    '108-d_tuna_fish_can',
    '108-e_tuna_fish_can',
    '109-a_soymilk',
    '109-b_soymilk',
    '109-c_soymilk',
    '109-d_soymilk',
    '109-e_soymilk',
    '110-a_coffee_can',
    '110-b_coffee_can',
    '110-c_coffee_can',
    '110-d_coffee_can',
    '110-e_coffee_can'
]

In [21]:
ViTac = ViTacData('/datasets/eventdata/batch2/', 'bd_data/bd_002_grasp_lift_hold/', list_of_objects2, selection="grasp_lift_hold")

In [22]:
%%time
ViTac.binarize_save(bin_duration=0.02, modality=2)

107-a_pepsi_bottle  being processed ...
107-b_pepsi_bottle  being processed ...
107-c_pepsi_bottle  being processed ...
107-d_pepsi_bottle  being processed ...
107-e_pepsi_bottle  being processed ...
108-a_tuna_fish_can  being processed ...
108-b_tuna_fish_can  being processed ...
108-c_tuna_fish_can  being processed ...
108-d_tuna_fish_can  being processed ...
108-e_tuna_fish_can  being processed ...
109-a_soymilk  being processed ...
109-b_soymilk  being processed ...
109-c_soymilk  being processed ...
109-d_soymilk  being processed ...
109-e_soymilk  being processed ...
110-a_coffee_can  being processed ...
110-b_coffee_can  being processed ...
110-c_coffee_can  being processed ...
110-d_coffee_can  being processed ...
110-e_coffee_can  being processed ...
Tactile has done
CPU times: user 25.1 s, sys: 2.04 s, total: 27.1 s
Wall time: 19min 10s


In [23]:
dsfsd

NameError: name 'dsfsd' is not defined

In [ ]:
test_labels = {'half_split':[], 
              '80_20_1':[],
              '80_20_2':[],
              '80_20_3':[],
              '80_20_4':[],
              '80_20_5':[]}
train_labels = {'half_split':[], 
              '80_20_1':[],
              '80_20_2':[],
              '80_20_3':[],
              '80_20_4':[],
              '80_20_5':[]}
current_label = -1
overall_count = -1
for obj in list_of_objects2:
    current_label += 1
    for i in range(0, 20):
        overall_count+=1
        if i in [1,3,5,7,9,11,13,15,17,19]:
            test_labels['half_split'].append([overall_count, current_label])
        else:
            train_labels['half_split'].append([overall_count, current_label])
            
        if i in [3,5,14,17]:
            test_labels['80_20_1'].append([overall_count, current_label])
        else:
            train_labels['80_20_1'].append([overall_count, current_label])
            
        if i in [2,6,15,16]:
            test_labels['80_20_2'].append([overall_count, current_label])
        else:
            train_labels['80_20_2'].append([overall_count, current_label])
            
        if i in [1,3,11,19]:
            test_labels['80_20_3'].append([overall_count, current_label])
        else:
            train_labels['80_20_3'].append([overall_count, current_label])
            
        if i in [4,8,13,18]:
            test_labels['80_20_4'].append([overall_count, current_label])
        else:
            train_labels['80_20_4'].append([overall_count, current_label])
            
        if i in [7,9,12,17]:
            test_labels['80_20_5'].append([overall_count, current_label])
        else:
            train_labels['80_20_5'].append([overall_count, current_label])

In [ ]:
test_labels['80_20_4']

In [ ]:
for split in train_labels:
    np.random.shuffle(train_labels[split])
    #np.savetxt(ViTac.save_dir + 'train_' + split + '.txt', np.array(train_labels[split], dtype=int), fmt='%d', delimiter='\t')

for split in test_labels:
    np.random.shuffle(test_labels[split])
    #np.savetxt(ViTac.save_dir + 'test_' + split + '.txt', np.array(test_labels[split], dtype=int), fmt='%d', delimiter='\t')

In [ ]:
#np.savetxt(ViTac.save_dir + "train_temp_50.txt", np.array(train_labels, dtype=int), fmt='%d', delimiter='\t')
#np.savetxt(ViTac.save_dir + "test_temp_50.txt", np.array(test_labels, dtype=int), fmt='%d')

In [ ]:
#np.savetxt(ViTac.save_dir + "train_temp_50.txt", np.array(train_labels, dtype=int), fmt='%d', delimiter='\t')
#np.savetxt(ViTac.save_dir + "test_temp_50.txt", np.array(test_labels, dtype=int), fmt='%d')

In [ ]:
#ViTac.save_dir

In [ ]:
#list_of_objects[20]

In [ ]:
aaa = np.array(new_train_labels[split])